In [1]:
import cv2
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
from math import ceil
from skimage import feature

In [ ]:
class CoringDetector:
    def apply_transforms(
        self,
        images: np.array,
        canny_sigma: float = 2.7,
        N1: int = 19,
        N2: int = 9,
        N3: int = 5,
    ):
        # Calcular bosdas com canny
        images = [feature.canny(img, sigma=canny_sigma) * 255.0 for img in images]

        # Fechar bordas encontradas com fechamento (para criar blobs)
        kernel1 = np.ones((N1, N1), np.uint8)
        images = [cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel1) for img in images]

        # Aplicando abertura com kernel vertical para remover ruidos horizontais
        kernel2 = np.ones((N2, 1), np.uint8)
        images = [cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel2) for img in images]

        # Aplicando dilatação para juntar ruidos proximos em grandes regiões, pode facilitar
        # na filtragem por área, mas talvez não seja necessario.
        kernel3 = np.ones((N3, N3), np.uint8)
        images = [cv2.morphologyEx(img, cv2.MORPH_DILATE, kernel3) for img in images]


    def find_contours(self, images: list):

        converted_images = []
        for image in np.array(images):
            # Ensure that the image is in uint8 format (CV_8U)
            if image.max() == 1:
                image *= 255
            if image.dtype != np.uint8:
                image = image.astype(np.uint8)
            # Convert to CV_8UC1 format
            if len(image.shape) > 2:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            converted_images.append(image)
        
        contours = [ contourn ]
        for img in converted_images:

            contours_list,_ = cv2.findContours(img, cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_NONE)
            contours.append(contours_list)

    def apply_thresholds(self, contours: np.array):
        ...